In [154]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [155]:
DATAPATH = './data/games.csv'

In [156]:
games = pd.read_csv(DATAPATH)

In [157]:
games.dtypes

Unnamed: 0             int64
SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [158]:
games = games.drop(['Unnamed: 0'], axis=1)

In [159]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612748,MIA,Miami Heat,21900970,2020-03-11,MIA vs. CHA,L,239,98,...,0.833,9,27,36,32,8,5,14,17,-13.0
1,22019,1610612743,DEN,Denver Nuggets,21900973,2020-03-11,DEN @ DAL,L,240,97,...,0.625,3,37,40,23,6,4,15,20,-7.8
2,22019,1610612752,NYK,New York Knicks,21900969,2020-03-11,NYK @ ATL,W,264,136,...,0.735,6,41,47,32,14,9,12,23,0.0
3,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,W,240,113,...,0.739,13,39,52,23,9,2,10,14,8.0
4,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,L,241,106,...,0.680,6,26,32,25,10,3,8,24,-14.2


In [160]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('WL')
    ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [161]:
games.pop('SEASON_ID')
games.pop('TEAM_ID')
games.pop('GAME_ID')
games.pop('TEAM_ABBREVIATION')

0        MIA
1        DEN
2        NYK
3        DAL
4        DET
5        CHA
6        PHI
7        ATL
8        GSW
9        PHX
10       DAL
11       POR
12       LAC
13       IND
14       HOU
15       SAS
16       ORL
17       BOS
18       NYK
19       WAS
20       CLE
21       BKN
22       LAL
23       MEM
24       CHI
25       MIN
26       CHA
27       MIL
28       ATL
29       UTA
30       DEN
31       TOR
32       DAL
33       MIL
34       TOR
35       IND
36       DET
37       SAS
38       BKN
39       CHI
40       ORL
41       CLE
42       HOU
43       MIN
44       SAC
45       NYK
46       BOS
47       LAC
48       MIA
49       WAS
50       OKC
51       NOP
52       PHX
53       LAL
54       HOU
55       POR
56       CLE
57       DEN
58       UTA
59       DET
60       CHA
61       SAC
62       ATL
63       PHI
64       GSW
65       MEM
66       NYK
67       CHI
68       MIN
69       UTA
70       NOP
71       BKN
72       ATL
73       MEM
74       LAL
75       SAS
76       IND

In [132]:
feature_columns = []
obj_columns = ['TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL']
#numeric columns
for i in obj_columns:
    games[i] = pd.Categorical(games[i])
    games[i] = games[i].cat.codes 
for header in games.columns:
    games[header] = games[header].astype('float64')
    if header != 'WL':
        feature_columns.append(feature_column.numeric_column(header))

In [133]:
games.dtypes

TEAM_NAME     float64
GAME_DATE     float64
MATCHUP       float64
WL            float64
MIN           float64
PTS           float64
FGM           float64
FGA           float64
FG_PCT        float64
FG3M          float64
FG3A          float64
FG3_PCT       float64
FTM           float64
FTA           float64
FT_PCT        float64
OREB          float64
DREB          float64
REB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PLUS_MINUS    float64
dtype: object

In [134]:
games.head(n=250)

,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42.0,2427.0,1050.0,0.0,239.0,98.0,39.0,84.0,0.464,15.0,...,0.833,9.0,27.0,36.0,32.0,8.0,5.0,14.0,17.0,-13.0
1,22.0,2427.0,465.0,0.0,240.0,97.0,39.0,83.0,0.470,14.0,...,0.625,3.0,37.0,40.0,23.0,6.0,4.0,15.0,20.0,-7.8
2,52.0,2427.0,1395.0,1.0,264.0,136.0,50.0,94.0,0.532,11.0,...,0.735,6.0,41.0,47.0,32.0,14.0,9.0,12.0,23.0,0.0
3,21.0,2427.0,434.0,1.0,240.0,113.0,42.0,92.0,0.457,12.0,...,0.739,13.0,39.0,52.0,23.0,9.0,2.0,10.0,14.0,8.0
4,23.0,2427.0,546.0,0.0,241.0,106.0,39.0,84.0,0.464,11.0,...,0.680,6.0,26.0,32.0,25.0,10.0,3.0,8.0,24.0,-14.2
5,18.0,2427.0,215.0,1.0,239.0,109.0,37.0,82.0,0.451,17.0,...,0.818,15.0,35.0,50.0,20.0,7.0,4.0,14.0,9.0,12.2
6,57.0,2427.0,1642.0,1.0,241.0,124.0,42.0,83.0,0.506,14.0,...,0.813,13.0,37.0,50.0,28.0,7.0,4.0,13.0,18.0,14.0
7,2.0,2427.0,54.0,0.0,265.0,131.0,48.0,106.0,0.453,14.0,...,0.808,15.0,38.0,53.0,26.0,6.0,3.0,17.0,25.0,0.0
8,28.0,2426.0,644.0,0.0,239.0,107.0,37.0,79.0,0.468,11.0,...,0.815,4.0,31.0,35.0,25.0,3.0,0.0,9.0,17.0,-24.0
9,58.0,2426.0,1692.0,0.0,242.0,105.0,35.0,87.0,0.402,11.0,...,0.960,9.0,34.0,43.0,26.0,3.0,5.0,14.0,16.0,-7.2


In [135]:
games = games.dropna()

In [136]:
print(feature_columns)

[NumericColumn(key='TEAM_NAME', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GAME_DATE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MATCHUP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MIN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG_PCT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3M', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3A', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key=

In [137]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [138]:
train, test = train_test_split(games, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

19196 train examples
4800 validation examples
6000 test examples


In [139]:
batch_size = 128
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [152]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='relu')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [153]:
model = get_compiled_model()
model.fit(train_ds, epochs=20)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

150/150 [==============================] - 0s 2ms/step - loss: 0.6617 - accuracy: 0.5986
Epoch 2/20
150/150 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.8073
Epoch 3/20
150/150 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.9390
Epoch 4/20
150/150 [==============================] - 0s 2ms/step - loss: 0.3873 - accuracy: 0.9694
Epoch 5/20
150/150 [==============================] - 0s 2ms/step - loss: 0.3811 - accuracy: 0.9734
Epoch 6/